#  Flood inference demonstrator
This notebook is here to show the different steps to follow to make an inference with the FloodML Random Forest algorithm. We focused on the Sentinel-1 inferences, as they represent the immense majority of inferences that have been done and studied in the frame of the FloodML project. Fourinferences can be launched in the frame of the demonstrator:

  * EMSR 267 in Lituania, tile 34UEG, date 20180202, orbit DES_051
  * EMSR 271 in Greece, tile 34SEJ, date 20180228, orbit DES_080
  * EMSR 279 in Spain, tile 30TXM, date 20180413, orbit ASC_030
  * EMSR 352 in Iran, tile 39RTQ, date 20190408, orbit DES_108
  
For the purpose of this notebook the slope files derived from MERIT DEM have already been prepared. The inference will be created in the "HomemadeINF" directory of the demonstrator, to seperate them from the inferences already put in the "INFERENCES directory", with purpose only for diplay.

To launch one of the inferences from this notebook, please fill properly the variables in the next cell.

BEWARE: the static display might not work if you are not on a Batch full node 24-cores and 120Gb Kernel. The inference in this frame take between 30min and an hour.

## Calculation parameters

In [5]:

tile = "39RTQ" # String containing the Sentinel-2 tile number 
date = "20190408" # String of the chosen date
orbit = "DES_108" # String of the chosen orbit
###thresh = 60 # int/float for the thresholding of water occurrences in GSWO product. 60 is set as default 

print('Tile ref: ', tile)
print('Date : ', date)
print('Orbit : ', orbit)

# Configuration file
pconf = 'config.json' 

Tile ref:  39RTQ
Date :  20190408
Orbit :  DES_108


## Loading of the needed libraries and the configuration file

In [8]:
import os, glob, sys, logging, argparse, shutil, gc, joblib
from distutils.dir_util import copy_tree
from osgeo import ogr
import lxml.etree as ET
import numpy as np
from tqdm import tqdm
import Demo_tools as dtool

conf = dtool.applyconf(pconf)

root_dir = conf['root_dir']
sys.path.append(conf['pykic'])
print(sys.path)

import raster.pykic_gdal as rpg


ModuleNotFoundError: No module named 'miscellaneous'

##  Sentinel-1 stack creation
Joins S1 VV and VH images along with MERIT-derived slopes in a stack for the inference.

In [3]:
#Loading and normalizing the slope file
slp_file = glob.glob(os.path.join(root_dir, 'Ancillary', 'MERIT_SLP', tile+'_SLP.tif'))[0]
slp, proj, dim, tr = rpg.gdal2array(slp_file)
slp_norm = slp / 90  # Normalization
slp_norm[ (slp_norm <= 0)] = 0.01 # To avoid planar overdetection (slp=0 and nodata values set to 0.01)

# Loading the Sentinel-1 files
filename = glob.glob(os.path.join(root_dir, '**/**', 's1*'+tile+'*vv*'+orbit+'*.tif'), recursive=True)[0]  
print("\tFile on which inference will be done: ", filename)

S1VV, proj, dim, tr = rpg.gdal2array(filename)
S1VH, proj, dim, tr = rpg.gdal2array(filename.replace('vv', 'vh'))

S1VV = np.float32(S1VV)
S1VH = np.float32(S1VH)
S1VV[S1VV == 0] = np.nan
S1VH[S1VH == 0] = np.nan

Vstack_S1 = np.hstack((np.reshape(S1VV, (-1, 1)), np.reshape(S1VH, (-1, 1))))
Vstack_S1 = np.hstack((Vstack_S1, np.reshape(slp_norm, (-1, 1))))


	File on which inference will be done:  /work/OT/floodml/data/deliveries/phase-2-cls/Demonstrateur/EMSR352/39RTQ/s1b_39RTQ_vv_DES_108_20190408txxxxxx.tif


##  Inference with Random Forest

In [4]:
DBpath = os.path.join(root_dir, 'DB_RDF_S1.sav')

# Remove NaN & predict
vec_ok = np.ravel(np.flatnonzero(np.sum(np.isnan(Vstack_S1) * 1, axis=1) == 0))
yb = np.array([])
yb = Vstack_S1[vec_ok]

# RANDOM FOREST
print('Loading RDF model...')
rdf = joblib.load(DBpath)  # path to be changed

print("Inference beginning")
rdf_pred = rdf.predict(yb)

print('Inference done')


	yb 120560285 3
	Loading RDF model...
	Inference beginning
	Inference done.


##  Inference exportation

In [6]:
# Output image size determiantion
MAT, proj, dim, tr = rpg.gdal2array(filename)
exout = np.zeros((dim[1], dim[0]), dtype=np.uint8)
exout[np.unravel_index(vec_ok, (dim[1], dim[0]))] = rdf_pred

# Output export
nexout = os.path.join(root_dir, 'HomemadeINF', 'Inference_RDF_S1_T'+tile+'_'+date+'_'+orbit+'_0_01nan.tif')
rpg.array2tif(nexout, exout, proj, dim, tr)